# **CART**

CART is not the same as ID3.

CART typically performs binary splits. For continuous features, it finds the optimal threshold to split the data, creating binary splits. It also handle categorical features natively. 

CART uses Gini impurity for classification problems. For pruning, it includes a cost-complexity pruning method that allows for post-pruning to improve generalization and avoid overfitting. 

## **Representation**

Note: entropy or training error can also be used as the splitting criterion, but it's more common to use Gini impurity.

$$\text{CART}(S, A)$$
$\text{Inputs: training set }S\text{, feature subset }A\subseteq [d]$<br>
$\textbf{if }\text{all examples in }S\text{ are labeled by }1\text{, return a leaf }1$<br>
$\textbf{if }\text{all examples in }S\text{ are labeled by }0\text{, return a leaf }0$<br>
$\textbf{if }A=\emptyset\text{, return a leaf whose value }=\text{ majority of labels in }S$<br>
$\textbf{else}\text{:}$<br>
$\quad \textbf{foreach}\text{ feature }i\text{ in }A\text{:}$<br>
$\quad\quad\textbf{if }x_i\text{ is continuous: }$<br>
$\quad\quad\quad\text{Generate a sequence of thresholds }\theta \text{ based on the sorted values of }x_i$<br>
$\quad\quad\quad\textbf{foreach}\text{ threshold }\theta\text{ in the sequence:}$<br>
$\quad\quad\quad\quad S_1 = \{(\textbf{x},y)\in S:x_i\le\theta\}$<br>
$\quad\quad\quad\quad S_2 = \{(\textbf{x},y)\in S:x_i>\theta\}$<br>
$\quad\quad\quad\quad \text{Gini}(S,i)=\frac{|S_1|}{|S|}\cdot\text{Gini}(S_1)+\frac{|S_2|}{|S|}\cdot\text{Gini}(S_2)$<br>
$\quad\quad\textbf{if }x_i\text{ is categorical: }$<br>
$\quad\quad\quad\text{Let categories }= \text{ unique values of }x_i$<br>
$\quad\quad\quad\textbf{foreach}\text{ binary split }(a,b)\text{ of categories:}$<br>
$\quad\quad\quad\quad S_1 = \{(\textbf{x},y)\in S:x_i\in a\}$<br>
$\quad\quad\quad\quad S_2 = \{(\textbf{x},y)\in S:x_i\in b\}$<br>
$\quad\quad\quad\quad \text{Gini}(S,i)=\frac{|S_1|}{|S|}\cdot\text{Gini}(S_1)+\frac{|S_2|}{|S|}\cdot\text{Gini}(S_2)$<br>
$\quad \text{Let } j = \text{argmin}_{i\in A}\text{Gini}(S,i)$<br>
$\quad \textbf{if }j \text{ is continuous:}$<br>
$\quad\quad S_1 = \{(\textbf{x},y)\in S:x_j\le\theta\}$<br>
$\quad\quad S_2 = \{(\textbf{x},y)\in S:x_j>\theta\}$<br>
$\quad \textbf{if }j \text{ is categorical:}$<br>
$\quad\quad S_1 = \{(\textbf{x},y)\in S:x_j\in a\}$<br>
$\quad\quad S_2 = \{(\textbf{x},y)\in S:x_j\in b\}$<br>
$\quad\text{Let }T_1\text{ be the tree returned by CART}(S_1,A\backslash\{j\})$<br>
$\quad\text{Let }T_2\text{ be the tree returned by CART}(S_2,A\backslash\{j\})$<br>
$\quad\text{Return the tree}$


## **Loss**
$$\text{Gini}(S)=1-\sum^K_{k=1}p_k^2$$
We split two subsets namely $S_1$ and $S_2$, and we have the Gini impurity of $S_1$ and $S_2$ as:
$$\text{Gini}(S_1)=1-\sum^K_{k=1}p_k^2$$
$$\text{Gini}(S_2)=1-\sum^K_{k=1}p_k^2$$
Where:
- $p_k$ is the proportion of samples in the subset that belong to class $k$.
- $K$ is the number of unique labels in the dataset.

In a binary classification problem, $K=2$ and we have only $p_1$ and $p_2$. Then:
$$Gini(S)=1-(p_1^2+p_2^2)$$
Letting $a=p_1$, we have:
$$\text{Gini}(S)=1-(a^2+(1-a)^2)=2a(1-a)$$
After splitting, the weighted Gini impurity is calculated as:
$$\text{Gini}(S,i)=\frac{|S_1|}{|S|}\cdot\text{Gini}(S_1)+\frac{|S_2|}{|S|}\cdot\text{Gini}(S_2)$$

## **Optimizer**

We will use pruning to avoid overfitting. Pruning in CART is done using cost-complexity pruning, where a penalty term is added.

The cost-complexity function for a tree $T$ is:
$$R_\alpha(T)=R(T)+\alpha\cdot|T|$$
Where:

- $R(T)$ is the impurity or misclassification cost of the tree on $S$.
- $|T|$ is the number of leaves in the tree.
- $\alpha > 0$ is a regularization parameter.

We can use metrics like Gini impurity, entropy, or training error to calculate $R(T)$. For any given tree, $R(T)$ is the sum of the misclassification costs of all leaves in the tree. For each leaf $t$ in $T$, the misclassification cost $R(t)$ is defined as:
$$R(t)=\frac{1}{N_t}\sum_{i\in t}\ell(y_i,\hat y_{i,t})$$
Where:
-  $N_t$ is the number of samples in leaf $t$
- $\ell(y_i,\hat y_{i,t})$ is the loss for one sample, where $y_i$ is the true label and $\hat y_{i,t}$ is the predicted label for leaf $t$.

For simplicity, we will use the 0-1 loss as the loss function here.
$$
\ell(y_i, \hat{y}_t) = \begin{cases} 
1, & \text{if } y_i \neq \hat{y}_{i,t} \\ 
0, & \text{if } y_i = \hat{y}_{i,t} 
\end{cases}
$$

Then, $R(T)$ is defined as:
$$R(T)=\sum_{t\in\text{leaves}(T)}R(t)$$
Where $\text{leaves}(T)$ is the set of all leaves in $T$.

We will minimize $R_\alpha(T)$ to find the one that best balances between complexity and loss and use cross-validation to select the best $\alpha$.
